In [ ]:
import cv2
import os
import numpy as np
from pygame import mixer
# import time
import tensorflow as tf
# from tensorflow.python.keras.models import save_model, load_model
# from keras.models import load_model

Initialize mixer for alarm sound

In [ ]:
mixer.init()
try:
    sound = mixer.Sound('alarm.wav')
except:
    print("Alarm file not found. Please ensure 'alarm.wav' is in the same directory.")
    sound = None

Load Haarcascade classifiers

In [ ]:
face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')

Load the trained model

In [ ]:
model = tf.keras.models.load_model('bestModel.keras')

Check if the model loaded successfully

In [ ]:
print(type(model))
model.summary()

Initialize variables

In [ ]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count = 0
score = 0
thicc = 2
lbl = ['Closed', 'Open']

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error accessing the webcam. Please check your camera.")
        break

    # Preprocess frame
    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces and eyes
    faces = face.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(25, 25))
    left_eye = leye.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))
    right_eye = reye.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (100, 100, 100), 1)

    # Process right eye
    rpred = 0
    for (x, y, w, h) in right_eye:
        r_eye = gray[y:y + h, x:x + w]
        r_eye = cv2.resize(r_eye, (64, 64))
        r_eye = r_eye / 255.0
        r_eye = np.expand_dims(r_eye, axis=-1)
        r_eye = np.expand_dims(r_eye, axis=0)
        prediction = model.predict(r_eye)
        rpred = 1 if prediction > 0.5 else 0
        break

    # Process left eye
    lpred = 0
    for (x, y, w, h) in left_eye:
        l_eye = gray[y:y + h, x:x + w]
        l_eye = cv2.resize(l_eye, (64, 64))
        l_eye = l_eye / 255.0
        l_eye = np.expand_dims(l_eye, axis=-1)
        l_eye = np.expand_dims(l_eye, axis=0)
        prediction = model.predict(l_eye)
        lpred = 1 if prediction > 0.5 else 0
        break

    # Determine status of eyes
    if rpred == 0 and lpred == 0:
        score += 1
        status = "Closed"
    else:
        score -= 1
        status = "Open"
    score = max(0, score)  # Prevent negative scores

    # Display status and score
    cv2.putText(frame, f"Status: {status}", (10, height - 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, f"Score: {score}", (10, height - 10), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

    # Trigger alarm if score exceeds threshold
    if score > 25:
        if sound and not mixer.get_busy():
            sound.play()
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)
        thicc = thicc + 2 if thicc < 16 else thicc - 2
        thicc = max(2, thicc)
    else:
        if sound:
            sound.stop()

    # Display frame
    cv2.imshow('Drowsiness Detection', frame)

    # Break loop on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Release resources